# Extract Topics and Short desc

In [42]:
!pip install openai langchain 


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
!pip install tiktoken


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
# LangChain basics
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain



# Langchain Loaders:
from langchain.document_loaders import YoutubeLoader
from langchain.document_loaders import WebBaseLoader

# Vector Store and retrievals
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
#import pinecone

# Chat Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Supporting libraries
import os
from dotenv import load_dotenv

load_dotenv()

True

In [45]:
!pip install bs4


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:

llm3 = ChatOpenAI(temperature=0,
                  model_name="gpt-3.5-turbo-0613",
                  request_timeout = 180
                )


In [47]:

website_loader = WebBaseLoader(["https://www.sciencedaily.com/news/computers_math/artificial_intelligence/", "https://news.mit.edu/topic/artificial-intelligence2"])
website_data = website_loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=2200)
website_docs = text_splitter.split_documents(website_data)
print (f"You have {len(website_docs)} docs. First doc is {llm3.get_num_tokens(website_docs[0].page_content)} tokens")

print(website_docs[0].metadata)

You have 1 docs. First doc is 1367 tokens
{'source': 'https://news.mit.edu/topic/artificial-intelligence2', 'title': 'Artificial intelligence | MIT News | Massachusetts Institute of Technology', 'language': 'en'}


In [48]:
!python -m pip install htmldate


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
from htmldate import find_date
import requests
html=requests.get('https://news.mit.edu/topic/artificial-intelligence2').content.decode('utf-8')
date = find_date(html)
print(date)

2023-09-14


In [50]:

# % START OF EXAMPLES
# - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
# - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
# - Revenge Against The Spam Calls: A couple of businesses focused on protecting consumers: RoboCall, TrueCaller, DoNotPay, FitIt
# - Wildcard CEOs vs. Prudent CEOs: However, Munger likes to surround himself with prudent CEO's and says he would never hire Musk.
# - Chess Business: Priyav, a college student, expressed his doubts on the MFM Facebook group about his Chess training business, mychesstutor.com, making $12.5K MRR with 90 enrolled.
# - Restaurant Refiller: An MFM Facebook group member commented on how they pay AirMark $1,000/month for toilet paper and toilet cover refills for their restaurant. Shaan sees an opportunity here for anyone wanting to compete against AirMark.
# - Collecting: Shaan shared an idea to build a mobile only marketplace for a collectors' category; similar to what StockX does for premium sneakers.
# % END OF EXAMPLES

template="""
You are a helpful assistant that helps retrieve distinct topics discussed from many websites' content
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics can include:
- AI tools
- GPT Models
- Google Models
- LLMs
- llama Models
- Falcon Models
- Programming Languages
- AI recent News
- AI tutorials
- OpenAI
- AI for business 
- AI for education
- AI for medicine 
- AI for art and music
- Deep Learning
- NLP
- Machine Learning
- Data science
- Opportunities in AI
- AI frameworks
- Future AI
- Langchain

- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the websites' content
- Do not respond with numbers, just bullet points of all topics listed under each other. Example:
Topics:
    - Topic 1 title: topic 1 description
    - Topic 2 title: topic 2 description
    - Topic 3 title: topic 3 description
    
- Ignore topics on policy and regulations
- Do not respond with anything outside of the webstes' content. If you can't extract any topics at all in the whole content, say 'Sorry, No topics found in the given content'
- Only pull topics from the websites' content. Do not use the examples
- If the authors' names were mentioned in the transcript, instead of saying 'The Author' refer to the names.
- Make your titles descriptive but concise. Example: 'Shaan's Experience at Twitch' should be 'Shaan's Interesting Projects At Twitch'
- A topic should be substantial, more than just a one-off comment

"""
#- Do not respond with anything outside of the transcript. If you don't see any topics, say, 'No Topics'
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Websites' Content: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_map = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

In [51]:
# % START OF EXAMPLES
# - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
# - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
# % END OF EXAMPLES

template="""
You are a helpful assistant that helps retrieve topics discussed in websites' content
- You will be given a series of bullet topics of topics found
- Your goal is to exract the topic names and brief 1-sentence description of the topic
- Do not respond with numbers, just bullet points of all topics listed under each other.
- Deduplicate any bullet points you see
- If you think two or more topics are similar and can be merged, merge them together with one topic title and create a new description that fits the merged topics
- Only pull topics from the websites' content. Do not use the examples.
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Websites' Content: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_combine = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])



In [52]:
chain = load_summarize_chain(llm3,
                             chain_type="map_reduce",
                             map_prompt=chat_prompt_map,
                             combine_prompt=chat_prompt_combine,
                            verbose=True
                            )

In [53]:
topics_found = chain.run({"input_documents": website_docs})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant that helps retrieve distinct topics discussed from many websites' content
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics can include:
- AI tools
- GPT Models
- Google Models
- LLMs
- llama Models
- Falcon Models
- Programming Languages
- AI recent News
- AI tutorials
- OpenAI
- AI for business 
- AI for education
- AI for medicine 
- AI for art and music
- Deep Learning
- NLP
- Machine Learning
- Data science
- Opportunities in AI
- AI frameworks
- Future AI
- Langchain

- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the websites' content
- Do not respond with numbers, just bullet points of all topics listed under each other. Example:
Topics:
    - Topic 1 title: topic 1 description
   

In [54]:
print(topics_found)

- Artificial intelligence: MIT News covers various news articles related to advancements in AI technology, applications of AI in different fields, and research on AI algorithms and models.
- Machine learning: MIT News features news articles on new algorithms, applications, and research in machine learning.
- Computer vision: MIT News covers articles on advancements in computer vision algorithms, applications in various industries, and research on improving computer vision systems.
- Robotics: MIT News features news articles on advancements in robotics technology, applications in different fields, and research on improving robotic systems.
- Data science: MIT News covers articles on new techniques and algorithms in data analysis, applications in various domains, and research on improving data science methods.
- AI in healthcare: MIT News features news articles on applications of AI in medical diagnosis, treatment planning, and healthcare management, as well as research on improving AI s

In [55]:
!pip install kor


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

In [71]:
schema = Object(
    id="topic",
    description="Topic Information",
    examples=[
        ("Artificial intelligence: MIT News covers advancements, applications, and research in AI technology and algorithms.", [{"title": "Artificial intelligence"}, {"description": "MIT News covers advancements, applications, and research in AI technology and algorithms."}, {"tag": "AI News"}]),
        ("Generative AI: An exciting tool in AI that can generate text based on input prompts.", [{"title": "Generative AI"}, {"description": "An exciting tool in AI that can generate text based on input prompts."}, {"tag": "AI Tools"}]),
        ("AI in relationships: The use of AI in relationship coaching and mentoring, such as the example of an AI-powered romantic relationship coaching app.", [{"title": "AI in relationships"}, {"description": "The use of AI in relationship coaching and mentoring, such as the example of an AI-powered romantic relationship coaching app."}, {"tag": "AI Applications"}]),
        ("AI in different industries: Exploring the potential of AI in various industries beyond consumer software and the internet.", [{"title": "AI in different industries"}, {"description": "Exploring the potential of AI in various industries beyond consumer software and the internet."}, {"tag": "AI Opportunities"}]),
        ("Supervised Learning: A technique in AI that is good at labeling things or computing input to outputs.", [{"title": "Supervised Learning"}, {"description": "A technique in AI that is good at labeling things or computing input to outputs."}, {"tag": "AI Tools"}]),
        ("Large Language Models: The power and potential of large language models in AI applications.", [{"title": "Large Language Models"}, {"description": "The power and potential of large language models in AI applications."}, {"tag": "AI LLMs"}]),
        ("Future growth of AI technologies: The prediction that supervised learning and generative AI will continue to grow in value and adoption over the next three years, with the potential for even greater expansion in the long term.", [{"title": "Future growth of AI technologies:"}, {"description": "The prediction that supervised learning and generative AI will continue to grow in value and adoption over the next three years, with the potential for even greater expansion in the long term."}, {"tag": "Future of AI"}]),
        ("Large Language Models: Dr. Andrew explains how large language models, like GPT, are built using supervised learning to predict the next word, enabling applications that can be built faster and more efficiently.", [{"title": "Large Language Models:"}, {"description": " Dr. Andrew explains how large language models, like GPT, are built using supervised learning to predict the next word, enabling applications that can be built faster and more efficiently."}, {"tag": "AI LLMs"}]),
 
    ],
    attributes=[
         Text(
            id="title",
            description="The title of the topic listed",
        ),
        Text(
            id="description",
            description="The description of the topic listed",
        ),
        Text(
            id="tag",
            description="The type of content being described",
        )
    ],
    many=True,
)

In [72]:
#chain = create_extraction_chain(schema, llm3)
chain = create_extraction_chain(llm3, schema)

In [73]:
topics_structured = chain.run(topics_found)

In [74]:
topics_structured

{'data': {'topic': [{'title': 'Artificial intelligence',
    'description': 'MIT News covers various news articles related to advancements in AI technology, applications of AI in different fields, and research on AI algorithms and models.',
    'tag': 'AI News'},
   {'title': 'Machine learning',
    'description': 'MIT News features news articles on new algorithms, applications, and research in machine learning.',
    'tag': 'AI News'},
   {'title': 'Computer vision',
    'description': 'MIT News covers articles on advancements in computer vision algorithms, applications in various industries, and research on improving computer vision systems.',
    'tag': 'AI News'},
   {'title': 'Robotics',
    'description': 'MIT News features news articles on advancements in robotics technology, applications in different fields, and research on improving robotic systems.',
    'tag': 'AI News'},
   {'title': 'Data science',
    'description': 'MIT News covers articles on new techniques and algorith

# Expand on Topics Found

In [75]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n", " "], chunk_size=4000, chunk_overlap=800)

docs= text_splitter.split_documents(website_data)

In [76]:
openai_embeddings = OpenAIEmbeddings(show_progress_bar=True, embedding_ctx_length=1024)

In [77]:
!pip install faiss-cpu


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
website_db = FAISS.from_documents(docs, openai_embeddings)

100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


In [79]:
# The system instructions. Notice the 'context' placeholder down below. This is where our relevant docs will go.
# The 'question' in the human message below won't be a question per se, but rather a topic we want to get relevant information on
system_template = """
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]

# This will pull the two messages together and get them ready to be sent to the LLM through the retriever
CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [80]:
from langchain.chains import RetrievalQA , RetrievalQAWithSourcesChain

In [81]:

# I'm also setting k=4 so the number of relevant docs we get back is 4. 
qa = RetrievalQA.from_chain_type(llm=llm3,
                                 chain_type="stuff",
                                 retriever=website_db.as_retriever(k=4),
                                 chain_type_kwargs = {
                                     'verbose': True,
                                     'prompt': CHAT_PROMPT
                                 },
                                return_source_documents=True)
                                 

In [82]:
topics_structured= topics_structured["data"]["topic"]

In [83]:
topics_structured

[{'title': 'Artificial intelligence',
  'description': 'MIT News covers various news articles related to advancements in AI technology, applications of AI in different fields, and research on AI algorithms and models.',
  'tag': 'AI News'},
 {'title': 'Machine learning',
  'description': 'MIT News features news articles on new algorithms, applications, and research in machine learning.',
  'tag': 'AI News'},
 {'title': 'Computer vision',
  'description': 'MIT News covers articles on advancements in computer vision algorithms, applications in various industries, and research on improving computer vision systems.',
  'tag': 'AI News'},
 {'title': 'Robotics',
  'description': 'MIT News features news articles on advancements in robotics technology, applications in different fields, and research on improving robotic systems.',
  'tag': 'AI News'},
 {'title': 'Data science',
  'description': 'MIT News covers articles on new techniques and algorithms in data analysis, applications in various 

In [84]:
for topic in topics_structured:
    query = f"""
        {topic['title']}: {topic['description']}
    """
    #expanded_topic = qa.run(query)
    expanded_topic = qa({"query": query})
    print(f"{topic['title']}: {topic['description']}")
    print ("\n\n")
    print(expanded_topic['result'])
    print ("\n\n")
    print(expanded_topic['source_documents'])
    print ("\n\n")

100%|██████████| 1/1 [00:00<00:00,  2.54it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



100%|██████████| 1/1 [00:00<00:00,  1.49it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



100%|██████████| 1/1 [00:00<00:00,  2.21it/s]


...



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



100%|██████████| 1/1 [00:00<00:00,  1.92it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



100%|██████████| 1/1 [00:00<00:00,  2.27it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



In [ ]:


# for topic in topics_structured[:1]:
#     query = f"""
#         {topic['title']}: {topic['description']}
#     """
#     #expanded_topic = qa.run(query)
#     expanded_topic = qa({"query": query})
#     print(f"{topic['title']}: {topic['description']}")
#     print ("\n\n")
#     print(expanded_topic['result'])
#     print ("\n\n")
#     print(type(expanded_topic['source_documents'][0]))  # document
#     print(type(expanded_topic['source_documents']))  #list
#     print ("\n\n")

#     for topic in expanded_topic['source_documents']:
#         print(type(topic)) #document
#         print(topic.metadata['source'])

    
        

# print(expanded_topic['source_documents'][0].metadata['source'])

# youtube_video_ID=expanded_topic['source_documents'][0].metadata['source']
# youtube_video_URL=f"https://www.youtube.com/watch?v={youtube_video_ID}"
# print(youtube_video_URL)

# #docs[0].metadata['source']